In [1]:
import os
import re
import glob
import numpy as np

# 설치 필요 시:
# !pip install nibabel
import nibabel as nib


SRC_DIR = "/SSD5_8TB/Daniel/14_fastddpm/Fast-DDPM/data/SynthradPelvis_train/MR"
DST_DIR = "/SSD5_8TB/Daniel/15_3dMedDiffusion/3D-MedDiffusion/data/SynthradPelvis_train/MR"
TARGET_SLICES = 128
PAD_VALUE = -1.0  # 최소값을 -1로 고정

os.makedirs(DST_DIR, exist_ok=True)

# 파일명 패턴: 1PA001_slice000.npy
pat = re.compile(r"^(?P<pid>.+)_slice(?P<idx>\d+)\.npy$")

def pad_or_crop_to_128(vol, target_slices=128, pad_value=-1.0):
    """
    vol shape: (H, W, S)  or (S, H, W) 중 하나여도 처리되도록 아래에서 통일.
    여기서는 (H, W, S)로 들어온다고 가정.
    """
    H, W, S = vol.shape
    if S == target_slices:
        return vol

    if S < target_slices:
        pad = target_slices - S
        pad_top = (pad + 1) // 2   # ceil
        pad_bottom = pad // 2      # floor

        top = np.full((H, W, pad_top), pad_value, dtype=vol.dtype)
        bottom = np.full((H, W, pad_bottom), pad_value, dtype=vol.dtype)
        out = np.concatenate([top, vol, bottom], axis=2)
        return out

    # S > target_slices: 가운데 기준 crop
    start = (S - target_slices) // 2
    end = start + target_slices
    return vol[:, :, start:end]


# 1) 모든 npy 파일 스캔
all_files = sorted(glob.glob(os.path.join(SRC_DIR, "*.npy")))

# 2) 환자별로 파일 그룹핑
by_patient = {}
for fp in all_files:
    base = os.path.basename(fp)
    m = pat.match(base)
    if not m:
        continue
    pid = m.group("pid")
    idx = int(m.group("idx"))
    by_patient.setdefault(pid, []).append((idx, fp))

print(f"Found patients: {len(by_patient)}")
print("Example patients:", list(by_patient.keys())[:5])

# 3) 환자별로 stack -> (H, W, S) 만들고 128 맞춘 뒤 nii.gz로 저장
saved = 0
skipped = 0

for pid, items in sorted(by_patient.items()):
    items = sorted(items, key=lambda x: x[0])  # slice index 정렬
    slice_paths = [p for _, p in items]

    # 첫 슬라이스로 shape 확인
    arr0 = np.load(slice_paths[0])
    if arr0.ndim != 2:
        print(f"[SKIP] {pid}: slice ndim={arr0.ndim} (expected 2D). path={slice_paths[0]}")
        skipped += 1
        continue

    H, W = arr0.shape

    # 모든 slice 로드
    slices = []
    bad = False
    for sp in slice_paths:
        a = np.load(sp)
        if a.shape != (H, W):
            print(f"[SKIP] {pid}: inconsistent shape {a.shape} != {(H,W)} at {sp}")
            bad = True
            break
        slices.append(a)

    if bad:
        skipped += 1
        continue

    # stack: (S, H, W) -> (H, W, S)
    vol = np.stack(slices, axis=0).transpose(1, 2, 0)

    # 128로 pad/crop
    vol_128 = pad_or_crop_to_128(vol, TARGET_SLICES, PAD_VALUE)

    # NIfTI 저장
    # affine은 단위행렬(메타가 없어서 기본값). 필요하면 spacing/방향 맞춰드릴 수 있음.
    affine = np.eye(4, dtype=np.float32)

    # nibabel은 보통 float32 권장
    vol_128 = vol_128.astype(np.float32)

    nii = nib.Nifti1Image(vol_128, affine)
    out_path = os.path.join(DST_DIR, f"{pid}.nii.gz")
    nib.save(nii, out_path)
    saved += 1

print(f"Done. saved={saved}, skipped={skipped}")
print("Output dir:", DST_DIR)


Found patients: 112
Example patients: ['1PA001', '1PA005', '1PA009', '1PA022', '1PA024']
Done. saved=112, skipped=0
Output dir: /SSD5_8TB/Daniel/15_3dMedDiffusion/3D-MedDiffusion/data/SynthradPelvis_train/MR


In [3]:
import os
import re
import glob
import numpy as np

# 설치 필요 시:
# !pip install nibabel
import nibabel as nib


SRC_DIR = "/SSD5_8TB/Daniel/14_fastddpm/Fast-DDPM/data/SynthradPelvis_test/MR"
DST_DIR = "/SSD5_8TB/Daniel/15_3dMedDiffusion/3D-MedDiffusion/data/SynthradPelvis_test/MR"
TARGET_SLICES = 128
PAD_VALUE = -1.0  # 최소값을 -1로 고정

os.makedirs(DST_DIR, exist_ok=True)

# 파일명 패턴: 1PA001_slice000.npy
pat = re.compile(r"^(?P<pid>.+)_slice(?P<idx>\d+)\.npy$")

def pad_or_crop_to_128(vol, target_slices=128, pad_value=-1.0):
    """
    vol shape: (H, W, S)  or (S, H, W) 중 하나여도 처리되도록 아래에서 통일.
    여기서는 (H, W, S)로 들어온다고 가정.
    """
    H, W, S = vol.shape
    if S == target_slices:
        return vol

    if S < target_slices:
        pad = target_slices - S
        pad_top = (pad + 1) // 2   # ceil
        pad_bottom = pad // 2      # floor

        top = np.full((H, W, pad_top), pad_value, dtype=vol.dtype)
        bottom = np.full((H, W, pad_bottom), pad_value, dtype=vol.dtype)
        out = np.concatenate([top, vol, bottom], axis=2)
        return out

    # S > target_slices: 가운데 기준 crop
    start = (S - target_slices) // 2
    end = start + target_slices
    return vol[:, :, start:end]


# 1) 모든 npy 파일 스캔
all_files = sorted(glob.glob(os.path.join(SRC_DIR, "*.npy")))

# 2) 환자별로 파일 그룹핑
by_patient = {}
for fp in all_files:
    base = os.path.basename(fp)
    m = pat.match(base)
    if not m:
        continue
    pid = m.group("pid")
    idx = int(m.group("idx"))
    by_patient.setdefault(pid, []).append((idx, fp))

print(f"Found patients: {len(by_patient)}")
print("Example patients:", list(by_patient.keys())[:5])

# 3) 환자별로 stack -> (H, W, S) 만들고 128 맞춘 뒤 nii.gz로 저장
saved = 0
skipped = 0

for pid, items in sorted(by_patient.items()):
    items = sorted(items, key=lambda x: x[0])  # slice index 정렬
    slice_paths = [p for _, p in items]

    # 첫 슬라이스로 shape 확인
    arr0 = np.load(slice_paths[0])
    if arr0.ndim != 2:
        print(f"[SKIP] {pid}: slice ndim={arr0.ndim} (expected 2D). path={slice_paths[0]}")
        skipped += 1
        continue

    H, W = arr0.shape

    # 모든 slice 로드
    slices = []
    bad = False
    for sp in slice_paths:
        a = np.load(sp)
        if a.shape != (H, W):
            print(f"[SKIP] {pid}: inconsistent shape {a.shape} != {(H,W)} at {sp}")
            bad = True
            break
        slices.append(a)

    if bad:
        skipped += 1
        continue

    # stack: (S, H, W) -> (H, W, S)
    vol = np.stack(slices, axis=0).transpose(1, 2, 0)

    # 128로 pad/crop
    vol_128 = pad_or_crop_to_128(vol, TARGET_SLICES, PAD_VALUE)

    # NIfTI 저장
    # affine은 단위행렬(메타가 없어서 기본값). 필요하면 spacing/방향 맞춰드릴 수 있음.
    affine = np.eye(4, dtype=np.float32)

    # nibabel은 보통 float32 권장
    vol_128 = vol_128.astype(np.float32)

    nii = nib.Nifti1Image(vol_128, affine)
    out_path = os.path.join(DST_DIR, f"{pid}.nii.gz")
    nib.save(nii, out_path)
    saved += 1

print(f"Done. saved={saved}, skipped={skipped}")
print("Output dir:", DST_DIR)


Found patients: 34
Example patients: ['1PA004', '1PA012', '1PA019', '1PA021', '1PA030']
Done. saved=34, skipped=0
Output dir: /SSD5_8TB/Daniel/15_3dMedDiffusion/3D-MedDiffusion/data/SynthradPelvis_test/MR


In [4]:
import os
import re
import glob
import numpy as np

# 설치 필요 시:
# !pip install nibabel
import nibabel as nib


SRC_DIR = "/SSD5_8TB/Daniel/14_fastddpm/Fast-DDPM/data/SynthradPelvis_train/CT"
DST_DIR = "/SSD5_8TB/Daniel/15_3dMedDiffusion/3D-MedDiffusion/data/SynthradPelvis_train/CT"
TARGET_SLICES = 128
PAD_VALUE = -1.0  # 최소값을 -1로 고정

os.makedirs(DST_DIR, exist_ok=True)

# 파일명 패턴: 1PA001_slice000.npy
pat = re.compile(r"^(?P<pid>.+)_slice(?P<idx>\d+)\.npy$")

def pad_or_crop_to_128(vol, target_slices=128, pad_value=-1.0):
    """
    vol shape: (H, W, S)  or (S, H, W) 중 하나여도 처리되도록 아래에서 통일.
    여기서는 (H, W, S)로 들어온다고 가정.
    """
    H, W, S = vol.shape
    if S == target_slices:
        return vol

    if S < target_slices:
        pad = target_slices - S
        pad_top = (pad + 1) // 2   # ceil
        pad_bottom = pad // 2      # floor

        top = np.full((H, W, pad_top), pad_value, dtype=vol.dtype)
        bottom = np.full((H, W, pad_bottom), pad_value, dtype=vol.dtype)
        out = np.concatenate([top, vol, bottom], axis=2)
        return out

    # S > target_slices: 가운데 기준 crop
    start = (S - target_slices) // 2
    end = start + target_slices
    return vol[:, :, start:end]


# 1) 모든 npy 파일 스캔
all_files = sorted(glob.glob(os.path.join(SRC_DIR, "*.npy")))

# 2) 환자별로 파일 그룹핑
by_patient = {}
for fp in all_files:
    base = os.path.basename(fp)
    m = pat.match(base)
    if not m:
        continue
    pid = m.group("pid")
    idx = int(m.group("idx"))
    by_patient.setdefault(pid, []).append((idx, fp))

print(f"Found patients: {len(by_patient)}")
print("Example patients:", list(by_patient.keys())[:5])

# 3) 환자별로 stack -> (H, W, S) 만들고 128 맞춘 뒤 nii.gz로 저장
saved = 0
skipped = 0

for pid, items in sorted(by_patient.items()):
    items = sorted(items, key=lambda x: x[0])  # slice index 정렬
    slice_paths = [p for _, p in items]

    # 첫 슬라이스로 shape 확인
    arr0 = np.load(slice_paths[0])
    if arr0.ndim != 2:
        print(f"[SKIP] {pid}: slice ndim={arr0.ndim} (expected 2D). path={slice_paths[0]}")
        skipped += 1
        continue

    H, W = arr0.shape

    # 모든 slice 로드
    slices = []
    bad = False
    for sp in slice_paths:
        a = np.load(sp)
        if a.shape != (H, W):
            print(f"[SKIP] {pid}: inconsistent shape {a.shape} != {(H,W)} at {sp}")
            bad = True
            break
        slices.append(a)

    if bad:
        skipped += 1
        continue

    # stack: (S, H, W) -> (H, W, S)
    vol = np.stack(slices, axis=0).transpose(1, 2, 0)

    # 128로 pad/crop
    vol_128 = pad_or_crop_to_128(vol, TARGET_SLICES, PAD_VALUE)

    # NIfTI 저장
    # affine은 단위행렬(메타가 없어서 기본값). 필요하면 spacing/방향 맞춰드릴 수 있음.
    affine = np.eye(4, dtype=np.float32)

    # nibabel은 보통 float32 권장
    vol_128 = vol_128.astype(np.float32)

    nii = nib.Nifti1Image(vol_128, affine)
    out_path = os.path.join(DST_DIR, f"{pid}.nii.gz")
    nib.save(nii, out_path)
    saved += 1

print(f"Done. saved={saved}, skipped={skipped}")
print("Output dir:", DST_DIR)


Found patients: 112
Example patients: ['1PA001', '1PA005', '1PA009', '1PA022', '1PA024']
Done. saved=112, skipped=0
Output dir: /SSD5_8TB/Daniel/15_3dMedDiffusion/3D-MedDiffusion/data/SynthradPelvis_train/CT


In [5]:
import os
import re
import glob
import numpy as np

# 설치 필요 시:
# !pip install nibabel
import nibabel as nib


SRC_DIR = "/SSD5_8TB/Daniel/14_fastddpm/Fast-DDPM/data/SynthradPelvis_test/CT"
DST_DIR = "/SSD5_8TB/Daniel/15_3dMedDiffusion/3D-MedDiffusion/data/SynthradPelvis_test/CT"
TARGET_SLICES = 128
PAD_VALUE = -1.0  # 최소값을 -1로 고정

os.makedirs(DST_DIR, exist_ok=True)

# 파일명 패턴: 1PA001_slice000.npy
pat = re.compile(r"^(?P<pid>.+)_slice(?P<idx>\d+)\.npy$")

def pad_or_crop_to_128(vol, target_slices=128, pad_value=-1.0):
    """
    vol shape: (H, W, S)  or (S, H, W) 중 하나여도 처리되도록 아래에서 통일.
    여기서는 (H, W, S)로 들어온다고 가정.
    """
    H, W, S = vol.shape
    if S == target_slices:
        return vol

    if S < target_slices:
        pad = target_slices - S
        pad_top = (pad + 1) // 2   # ceil
        pad_bottom = pad // 2      # floor

        top = np.full((H, W, pad_top), pad_value, dtype=vol.dtype)
        bottom = np.full((H, W, pad_bottom), pad_value, dtype=vol.dtype)
        out = np.concatenate([top, vol, bottom], axis=2)
        return out

    # S > target_slices: 가운데 기준 crop
    start = (S - target_slices) // 2
    end = start + target_slices
    return vol[:, :, start:end]


# 1) 모든 npy 파일 스캔
all_files = sorted(glob.glob(os.path.join(SRC_DIR, "*.npy")))

# 2) 환자별로 파일 그룹핑
by_patient = {}
for fp in all_files:
    base = os.path.basename(fp)
    m = pat.match(base)
    if not m:
        continue
    pid = m.group("pid")
    idx = int(m.group("idx"))
    by_patient.setdefault(pid, []).append((idx, fp))

print(f"Found patients: {len(by_patient)}")
print("Example patients:", list(by_patient.keys())[:5])

# 3) 환자별로 stack -> (H, W, S) 만들고 128 맞춘 뒤 nii.gz로 저장
saved = 0
skipped = 0

for pid, items in sorted(by_patient.items()):
    items = sorted(items, key=lambda x: x[0])  # slice index 정렬
    slice_paths = [p for _, p in items]

    # 첫 슬라이스로 shape 확인
    arr0 = np.load(slice_paths[0])
    if arr0.ndim != 2:
        print(f"[SKIP] {pid}: slice ndim={arr0.ndim} (expected 2D). path={slice_paths[0]}")
        skipped += 1
        continue

    H, W = arr0.shape

    # 모든 slice 로드
    slices = []
    bad = False
    for sp in slice_paths:
        a = np.load(sp)
        if a.shape != (H, W):
            print(f"[SKIP] {pid}: inconsistent shape {a.shape} != {(H,W)} at {sp}")
            bad = True
            break
        slices.append(a)

    if bad:
        skipped += 1
        continue

    # stack: (S, H, W) -> (H, W, S)
    vol = np.stack(slices, axis=0).transpose(1, 2, 0)

    # 128로 pad/crop
    vol_128 = pad_or_crop_to_128(vol, TARGET_SLICES, PAD_VALUE)

    # NIfTI 저장
    # affine은 단위행렬(메타가 없어서 기본값). 필요하면 spacing/방향 맞춰드릴 수 있음.
    affine = np.eye(4, dtype=np.float32)

    # nibabel은 보통 float32 권장
    vol_128 = vol_128.astype(np.float32)

    nii = nib.Nifti1Image(vol_128, affine)
    out_path = os.path.join(DST_DIR, f"{pid}.nii.gz")
    nib.save(nii, out_path)
    saved += 1

print(f"Done. saved={saved}, skipped={skipped}")
print("Output dir:", DST_DIR)


Found patients: 34
Example patients: ['1PA004', '1PA012', '1PA019', '1PA021', '1PA030']
Done. saved=34, skipped=0
Output dir: /SSD5_8TB/Daniel/15_3dMedDiffusion/3D-MedDiffusion/data/SynthradPelvis_test/CT
